| Información requerida                   | Información obtenida                                     |
|-----------------------------------------|----------------------------------------------------------|
| negocio                                 |                             |
| nombre cliente o razón social           |                       DESCRIPCION                        |
| número cliente                          |                       CLAVE                              |      
| estatus                                 | WHERE ESTATUS_GARANTIA IS NULL OR ESTATUS_GARANTIA = 'R' |
| número contrato                         |                      CLAVE DE GARANTIA                   |
| género                                  |                          |
| fecha_nacimiento o de constitución      |                          |
| entidad federativa nacimiento           |                          |
| país de nacimiento                      |                          |
| nacionalidad                            |                          |
| profesión                               |                          |
| calle                                   |                          |
| número_exterior                         |                          |
| número_interior                         |                          |
| colonia_urbanización                    |                          |
| delegación_municipio                    |                          |
| ciudad_población                        |                          |
| entidad_federativa                      |                          |
| código postal                           |                          |
| país                                    |                          |
| teléfono                                |             GIA_FUNCIONARIOS_FACULTADOS                  |
| correo_electrónico                      |                          |
| rfc                                     |                  CLAVE_INTERMEDIARIO_SIFC                |
| curp                                    |                          |
| fiel                                    |                          |
| representante legal                     |                          |
| tipo de persona                         |                          |    
| clasificación por grado de riesgo       |                          |
| pep (persona políticamente expuesta)    |                          |
| razón                                   |                          |
| producto contratado                     |                 GARANTIA: GIA.SIAG_PRODUCTOESPECIFICO  |
| cambios en perfil transaccional         |                          |


- SE TOMA EL ESTATUS DEL CLIENTE, CUANDO LA GARANTIA EN SU TABLA LA COLUMNA ESTATUS GARANTIA ESTÉ EN NULL O SEA 'R' REINSTALADA
- TELEFENO SE OBTIENE POR LA TABLA GIA_FUNCIONARIO INTER_CLAVE 
¿MONTO DEL CREDITO A GARANTIA?
¿PLAZO DEL CREDITO ?
¿TASA DE INTERES DEL CREDITO ? 
¿PORCENTAJE GARANTIZADO POR NAIFN?
¿MONTO GARANTIZADO POR NAIFN?
¿PORCENTAJE_COMISION'
MONTO_COMISION 
MONTO_CREDITO,  IMPORTE_DISPOSICION, PLAZO, GRACIA, TASA_INTERES, PORCENTAJE_GARANTIZADO, MONTO_GARANTIZADO, PORCENTAJE_COMISION,
CVE_CARTERA, CVE_PORTAFOLIO, 

In [35]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utils.clt_repetidos import *

In [24]:
df = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/Clientes_activos_SIAG.dat")

In [36]:
df["TELEFONO"] = df["TELEFONO"].apply(lambda x: " -- ".join(sorted(set(c.strip() for c in x.split("--") if c.strip() != 'NA'))) if "--" in str(x) else x)


In [37]:
df["RFC"] = df["RFC"].apply(lambda x: normalizar_RFC(x) if pd.notna(x) and verificar_rfc(normalizar_RFC(x)) else "XXXXXXXXXXXXX")

In [38]:
df['FECHA_DE_ACTUALIZACION'] = datetime.now().strftime('%d/%m/%y')
col_final = ['NEGOCIO', 'NOMBRE_O_RAZON_SOCIAL', 'NUMERO_CLIENTE', 'ESTATUS',
           'NUMERO_CONTRATO', 'GENERO', 'FECHA_NAC_O_CONST',
           'ENTIDAD_FEDERATIVA_NACIMIENTO', 'PAIS_DE_NACIMIENTO', 'NACIONALIDAD',
           'PROFESION', 'CALLE', 'NUMERO_EXTERIOR', 'NUMERO_INTERIOR',
           'COLONIA_URBANIZACION', 'DELEGACION_MUNICIPIO', 'CIUDAD_POBLACION',
           'ENTIDAD_FEDERATIVA', 'CODIGO_POSTAL', 'PAIS', 'TELEFONO',
           'CORREO_ELECTRONICO', 'RFC', 'CURP', 'FIEL', 'REPRESENTANTE_LEGAL',
           'TIPO_PERSONA', 'PRODUCTO_CONTRATADO',
           'SISTEMA_ORIGEN', 'FECHA_DE_ACTUALIZACION']
df = df[col_final]

In [28]:
clt_repetidos = compara_nombres(df, df, "NOMBRE_O_RAZON_SOCIAL", "NOMBRE_O_RAZON_SOCIAL","NUMERO_CLIENTE","NUMERO_CLIENTE", 95)
if not clt_repetidos.empty:
    df_resul, drops_ids = agregar_asociaciones_clientes(clt_repetidos,df)
    df = df.drop(drops_ids)
    df = pd.concat([df, df_resul], ignore_index=True)
    df = df.sort_values("NOMBRE_O_RAZON_SOCIAL")

In [39]:
tipo_financiera_sirac = {
    1: 'BANCOS',
    2: 'UNIONES DE CREDITO',
    3: 'ENTIDADES DE FOMENTO',
    4: 'EMP. DE FACTORAJE BANCARIAS',
    5: 'EMP. DE FACTORAJE NO BANCARIAS',
    6: 'ARRENDADORAS BANCARIAS',
    7: 'ARRENDADORAS NO BANCARIAS',
    8: 'AFIANZADORAS',
    9: 'FIDEICOMISO AAA',
    10: 'SOCIEDADES DE OBJETO LIMITADO',
    11: 'EMP. FACT. NO BANCARIAS PLUS',
    12: 'ARRENDAD. NO BANCARIAS PLUS',
    13: 'SOC. DE OBJETO LIMITADO PLUS',
    14: 'BANCOS 1ER. PISO',
    15: 'EMP FACT BANCARI C/CTA SOBGIRO',
    16: 'ORG. AUX. DE CREDITO 1ER. PISO',
    17: 'EMP. FACTORAJE 1ER. PISO',
    18: 'MANDATOS 1ER. PISO',
    19: 'SOC FIN OBJETO MULT 1PISO PLUS',
    20: 'FIDEICOMISOS 1ER. PISO',
    21: 'IFNB ESPECIALES',
    22: 'SOCIEDAD FINANCIERA POPULAR',
    23: 'SOC. FINAN. DE OBJETO MULTIPLE',
    24: 'FIDEICOMISO',
    25: 'ALMACENADORAS',
    26: 'DIRECTOS',
    27: 'COMISIONISTA',
    28: 'SINDICATO BANCARIO',
    29: 'SOC. COOPERATIVAS AHORRO PREST',
    30: 'SOCIEDAD ANONIMA PROMOTORA INV'
}

In [40]:
df["TIPO_PERSONA"] = df["TIPO_PERSONA"].apply(lambda x: tipo_financiera_sirac.get(x,x))

In [42]:
name_archivo = os.path.join("/home/ale1726/proyects/datalake/clientes/data/ETL","CLientesT_SIAG.csv")
df.to_csv(name_archivo,index=False)